# E8: Length × Cue × Corruption Experiment (FIXED VERSION)

**Paper**: A2 (Cue-Dominant Extraction Explains Length Effects)

**FIXES**:
1. Added L=15 (now L ∈ {5, 10, 15, 20})
2. Fixed cue removal - replaces ENTIRE final step

**Purpose**: Validate that length effect is CONDITIONAL on cue presence.

**Predictions**:
| Condition | L=5 | L=10 | L=15 | L=20 | Pattern |
|-----------|-----|------|------|------|---------|
| Cue-present | High | High | High | High | Flat (cue dominates) |
| Cue-absent | Low | Med | Med+ | High | Increasing (redundancy) |

**Expected inferences**: 188 × 4 × 2 × 2 = ~3,008

**Date**: 2026-01-03
**VERSION**: 2.0 (FIXED)

## 0. Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXPERIMENT_NAME = 'E8_length_cue_v2'
EXPERIMENT_DATE = datetime.now().strftime('%Y%m%d')

BASE_DIR = '/content/drive/MyDrive/CoT_Experiment'
V3_DATA_DIR = f'{BASE_DIR}/full_experiment_v3_20251224'
TRACES_DIR = f'{V3_DATA_DIR}/clean_traces'

SAVE_DIR = f'{BASE_DIR}/{EXPERIMENT_NAME}_{EXPERIMENT_DATE}'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f'{SAVE_DIR}/results', exist_ok=True)

print(f'Experiment: {EXPERIMENT_NAME}')
print(f'Save directory: {SAVE_DIR}')

## 1. Install Dependencies

In [ ]:
!pip install datasets anthropic matplotlib pandas tqdm scipy -q
print('Dependencies installed.')

## 2. Configuration

In [ ]:
import hashlib
import random
import json
import re
import time
from typing import List, Dict, Tuple, Optional, Any, Set
from dataclasses import dataclass, asdict, field
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# =============================================================================
# Global Configuration
# =============================================================================
GLOBAL_SEED = 20251224
E8_SEED = 20260103

# UPDATED: Now includes L=15
LENGTHS = [5, 10, 15, 20]
CORRUPTION_LEVELS = [0.4, 0.8]
CUE_CONDITIONS = ['present', 'absent']

CORRUPTION_RATIO = {'IRR': 1, 'LOC': 2, 'WRONG': 2}

API_MAX_TOKENS_ANSWER = 256
API_RETRY_DELAY = 1.0
API_RATE_LIMIT_DELAY = 0.5

print('='*70)
print('E8: LENGTH × CUE × CORRUPTION (FIXED VERSION)')
print('='*70)
print(f'  Lengths: {LENGTHS} (now includes L=15)')
print(f'  Corruption: {CORRUPTION_LEVELS}')
print(f'  Cue: {CUE_CONDITIONS}')
print(f'  Total conditions: {len(LENGTHS) * len(CORRUPTION_LEVELS) * len(CUE_CONDITIONS)}')
print('='*70)

## 3. Data Structures

In [ ]:
@dataclass
class GSM8KProblem:
    index: int
    question: str
    answer_text: str
    final_answer: int

@dataclass
class CleanTrace:
    problem_index: int
    I: int
    steps: List[str]
    full_text: str

@dataclass
class CorruptedTrace:
    problem_index: int
    L: int
    c: float
    cue_condition: str
    K_corrupt: int
    corrupted_steps: List[int]
    corruption_types: Dict[int, str]
    steps: List[str]
    full_text: str
    seed: int

@dataclass
class ExperimentResult:
    problem_index: int
    L: int
    c: float
    cue_condition: str
    K_corrupt: int
    K_clean: int
    model_answer: Optional[int]
    correct_answer: int
    is_correct: bool
    raw_output: str
    timestamp: str

## 4. Utility Functions

In [ ]:
def derive_seed(global_seed: int, problem_id: int, L: int, c: float, cue: str) -> int:
    key = f"{global_seed}|E8|{problem_id}|L={L}|c={c}|cue={cue}"
    h = hashlib.sha256(key.encode("utf-8")).hexdigest()
    return int(h[:8], 16)

def save_json(data: Any, filepath: str):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_json(filepath: str) -> Any:
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

## 5. Load Data

In [ ]:
# Load problems
problems_path = f'{V3_DATA_DIR}/problems_v3.json'
problems_data = load_json(problems_path)
problems = [GSM8KProblem(**p) for p in problems_data]
prob_map = {p.index: p for p in problems}
print(f'Loaded {len(problems)} problems')

# Load traces for each L
trace_maps = {}

for L in LENGTHS:
    trace_path = f'{TRACES_DIR}/clean_traces_I{L}_v3.json'
    if os.path.exists(trace_path):
        traces_data = load_json(trace_path)
        traces = [CleanTrace(**t) for t in traces_data]
        trace_maps[L] = {t.problem_index: t for t in traces}
        print(f'Loaded {len(traces)} traces for L={L}')
    else:
        print(f'ERROR: Traces for L={L} not found!')
        trace_maps[L] = {}

# Find common problems
common_indices = set.intersection(*[set(tm.keys()) for tm in trace_maps.values()])
print(f'\nCommon problems across L={LENGTHS}: {len(common_indices)}')

In [ ]:
experiment_problems = [p for p in problems if p.index in common_indices]
print(f'Using {len(experiment_problems)} problems')
print(f'Total inferences: {len(experiment_problems) * len(LENGTHS) * len(CORRUPTION_LEVELS) * len(CUE_CONDITIONS)}')

## 6. Corruption Logic

In [ ]:
def pick_corrupted_steps(L: int, c: float, seed: int) -> List[int]:
    """Select steps to corrupt (excluding final step)"""
    K = int(round(c * (L - 1)))  # Exclude last step
    if K == 0:
        return []
    steps = list(range(1, L))  # Steps 1 to L-1
    rng = random.Random(seed)
    rng.shuffle(steps)
    return sorted(steps[:K])

def assign_corruption_types(corrupted_steps: List[int], seed: int) -> Dict[int, str]:
    K = len(corrupted_steps)
    if K == 0:
        return {}
    n_irr = (K * 1) // 5
    n_loc = (K * 2) // 5
    n_wrong = K - n_irr - n_loc
    if n_wrong == 0 and K > 0:
        n_wrong = 1
        if n_loc > 0:
            n_loc -= 1
        elif n_irr > 0:
            n_irr -= 1
    rng = random.Random(seed + 1)
    perm = corrupted_steps[:]
    rng.shuffle(perm)
    type_map = {}
    for s in perm[:n_irr]:
        type_map[s] = "IRR"
    for s in perm[n_irr:n_irr + n_loc]:
        type_map[s] = "LOC"
    for s in perm[n_irr + n_loc:]:
        type_map[s] = "WRONG"
    return type_map

## 7. Corruption Templates

In [ ]:
IRRELEVANT_TEMPLATES = [
    "Compute an auxiliary value: aux = {a} + {b} = {result}, but it will not be used later.",
    "Compute a side quantity: aux = {a} * 2 = {result}, unrelated to the final result.",
    "Note that we can also compute aux = {a} - {b} = {result}, though this is not needed.",
]

WRONG_CONSTRAINT_TEMPLATES = [
    "Fix an intermediate condition: set {var} = {wrong_value} as a given constraint for the rest of the steps.",
    "Assume the total is {var} = {wrong_value} and proceed using this fixed value.",
]

def generate_irrelevant_step(step_num: int, seed: int) -> str:
    rng = random.Random(seed)
    a = rng.randint(2, 20)
    b = rng.randint(2, 20)
    template = rng.choice(IRRELEVANT_TEMPLATES)
    if '+' in template:
        result = a + b
    elif '*' in template:
        result = a * 2
    else:
        result = a - b
    return template.format(a=a, b=b, result=result)

def generate_local_error_step(original_step: str, seed: int) -> str:
    rng = random.Random(seed)
    numbers = re.findall(r'\d+', original_step)
    if not numbers:
        return f"Compute t = 10 * 3 = {rng.randint(28, 32)} (using the previous values)."
    original_result = int(numbers[-1])
    offset = rng.choice([-3, -2, -1, 1, 2, 3])
    wrong_result = max(0, original_result + offset)
    modified = re.sub(r'= (\d+)\.$', f'= {wrong_result}.', original_step)
    if modified == original_step:
        modified = re.sub(r'(\d+)\.$', f'{wrong_result}.', original_step)
    return modified

def generate_wrong_constraint_step(step_num: int, seed: int) -> str:
    rng = random.Random(seed)
    var = rng.choice(['x', 'total', 'result', 'n'])
    wrong_value = rng.randint(10, 100)
    template = rng.choice(WRONG_CONSTRAINT_TEMPLATES)
    return template.format(var=var, wrong_value=wrong_value)

## 8. FIXED: Final Step Manipulation

**BUG IN PREVIOUS VERSION**: Partial replacement left residual expressions.

**FIX**: Replace the ENTIRE final step.

In [ ]:
def create_final_step_with_cue(answer: int) -> str:
    """
    FIXED: Create a clean final step WITH the cue.
    Replaces the ENTIRE step to avoid residual expressions.
    """
    return f"Therefore, the final answer is Final = {answer}."

def create_final_step_without_cue() -> str:
    """
    FIXED: Create a clean final step WITHOUT the cue.
    Replaces the ENTIRE step to avoid residual expressions.
    """
    return "The reasoning steps above lead to the solution. The calculation is now complete."

# Verify
print("=== FIXED FINAL STEP TEMPLATES ===")
print(f"Cue present: {create_final_step_with_cue(70000)}")
print(f"Cue absent: {create_final_step_without_cue()}")
print("\nNo residual expressions like '- 130000' will remain.")

## 9. Trace Creation

In [ ]:
def create_corrupted_trace(
    clean_trace: CleanTrace,
    correct_answer: int,
    c: float,
    cue_condition: str,
    seed: int
) -> CorruptedTrace:
    """
    Create a corrupted trace with specified corruption level and cue condition.
    FIXED: Uses complete final step replacement.
    """
    L = clean_trace.I
    
    # Pick steps to corrupt (excluding final step)
    corrupted_steps = pick_corrupted_steps(L, c, seed)
    corruption_types = assign_corruption_types(corrupted_steps, seed)
    K_corrupt = len(corrupted_steps)
    
    # Apply corruption to steps 1 to L-1
    new_steps = []
    for i, step_content in enumerate(clean_trace.steps[:-1]):
        step_num = i + 1
        
        if step_num in corrupted_steps:
            ctype = corruption_types[step_num]
            step_seed = seed + step_num * 1000
            
            if ctype == 'IRR':
                new_content = generate_irrelevant_step(step_num, step_seed)
            elif ctype == 'LOC':
                new_content = generate_local_error_step(step_content, step_seed)
            else:
                new_content = generate_wrong_constraint_step(step_num, step_seed)
            new_steps.append(new_content)
        else:
            new_steps.append(step_content)
    
    # FIXED: Handle final step with complete replacement
    if cue_condition == 'present':
        final_step = create_final_step_with_cue(correct_answer)
    else:
        final_step = create_final_step_without_cue()
    new_steps.append(final_step)
    
    # Build full text
    lines = ['[[COT_START]]']
    for i, content in enumerate(new_steps):
        lines.append(f'Step {i+1}: {content}')
    lines.append('[[COT_END]]')
    full_text = '\n'.join(lines)
    
    return CorruptedTrace(
        problem_index=clean_trace.problem_index,
        L=L,
        c=c,
        cue_condition=cue_condition,
        K_corrupt=K_corrupt,
        corrupted_steps=corrupted_steps,
        corruption_types=corruption_types,
        steps=new_steps,
        full_text=full_text,
        seed=seed
    )

## 10. API Setup

In [ ]:
from getpass import getpass

ANTHROPIC_API_KEY = getpass('Enter Anthropic API Key: ')
print('API Key set.')

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def call_claude(system_prompt: str, user_prompt: str, max_tokens: int = 1024, retries: int = 3) -> str:
    for attempt in range(retries):
        try:
            message = client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=max_tokens,
                messages=[{"role": "user", "content": user_prompt}],
                system=system_prompt,
                temperature=0
            )
            time.sleep(API_RATE_LIMIT_DELAY)
            return message.content[0].text
        except Exception as e:
            print(f'API error (attempt {attempt+1}): {e}')
            if attempt < retries - 1:
                time.sleep(API_RETRY_DELAY * (attempt + 1))
            else:
                raise

test_response = call_claude(
    "You output ONLY JSON.",
    'Respond with exactly: {"test": "ok"}',
    max_tokens=50
)
print(f'API test: {test_response}')

## 11. Experiment Prompts

In [ ]:
EXPERIMENT_SYSTEM_PROMPT = """You are a calculator that outputs ONLY JSON.

CRITICAL RULES:
1. Your output MUST start with the character '{'
2. Your output MUST be exactly: {"final": <number>}
3. Replace <number> with an integer (the numerical answer)
4. Do NOT write ANY explanation, reasoning, or text before or after the JSON
5. Do NOT write "I need to" or "Let me" or any other words
6. ONLY output the JSON object, nothing else

CORRECT OUTPUT EXAMPLE:
{"final": 42}
"""

def create_experiment_prompt(problem: GSM8KProblem, cot_text: str) -> Tuple[str, str]:
    user = f"""Problem: {problem.question}

Reasoning trace (use these steps as given facts):
{cot_text}

Based on the trace above, compute the final numerical answer.
OUTPUT ONLY: {{"final": <number>}}
START YOUR RESPONSE WITH '{{'"""
    return EXPERIMENT_SYSTEM_PROMPT, user

def parse_model_answer(response: str) -> Optional[int]:
    match = re.search(r'\{\s*"final"\s*:\s*(-?\d+(?:\.\d+)?)\s*\}', response)
    if match:
        return int(round(float(match.group(1))))
    match = re.search(r"\{\s*[\"']final[\"']\s*:\s*(-?\d+(?:\.\d+)?)\s*\}", response)
    if match:
        return int(round(float(match.group(1))))
    match = re.search(r'"final"\s*:\s*(-?\d+(?:\.\d+)?)', response)
    if match:
        return int(round(float(match.group(1))))
    matches = re.findall(r'(?:^|\s)(-?\d+(?:\.\d+)?)(?:\s|$|\.|,)', response)
    if matches:
        return int(round(float(matches[-1])))
    return None

## 12. Run Experiment

In [ ]:
def run_single_experiment(
    problem: GSM8KProblem,
    trace: CorruptedTrace
) -> ExperimentResult:
    sys_prompt, usr_prompt = create_experiment_prompt(problem, trace.full_text)
    response = call_claude(sys_prompt, usr_prompt, max_tokens=API_MAX_TOKENS_ANSWER)
    
    model_answer = parse_model_answer(response)
    is_correct = (model_answer == problem.final_answer) if model_answer is not None else False
    
    return ExperimentResult(
        problem_index=problem.index,
        L=trace.L,
        c=trace.c,
        cue_condition=trace.cue_condition,
        K_corrupt=trace.K_corrupt,
        K_clean=trace.L - trace.K_corrupt,
        model_answer=model_answer,
        correct_answer=problem.final_answer,
        is_correct=is_correct,
        raw_output=response,
        timestamp=datetime.now().isoformat()
    )

In [ ]:
print('='*70)
print('E8: LENGTH × CUE × CORRUPTION (FIXED)')
print('='*70)

total_conditions = len(LENGTHS) * len(CORRUPTION_LEVELS) * len(CUE_CONDITIONS)
total_inferences = len(experiment_problems) * total_conditions
print(f'Problems: {len(experiment_problems)}')
print(f'Conditions: {total_conditions}')
print(f'Total inferences: {total_inferences}')
print('='*70)

all_results = []
traces_log = []

for L in LENGTHS:
    for c in CORRUPTION_LEVELS:
        for cue in CUE_CONDITIONS:
            condition_name = f'L={L}_c={c}_cue={cue}'
            print(f'\nRunning: {condition_name}')
            
            condition_results = []
            
            for prob in tqdm(experiment_problems, desc=condition_name):
                clean_trace = trace_maps[L].get(prob.index)
                if clean_trace is None:
                    continue
                
                seed = derive_seed(E8_SEED, prob.index, L, c, cue)
                corrupted_trace = create_corrupted_trace(clean_trace, prob.final_answer, c, cue, seed)
                result = run_single_experiment(prob, corrupted_trace)
                
                condition_results.append(result)
                all_results.append(result)
                
                if len(condition_results) <= 3:
                    traces_log.append({
                        'condition': condition_name,
                        'trace': asdict(corrupted_trace)
                    })
            
            acc = sum(r.is_correct for r in condition_results) / len(condition_results) if condition_results else 0
            print(f'  → Accuracy: {acc*100:.1f}% ({sum(r.is_correct for r in condition_results)}/{len(condition_results)})')

print(f'\n\nTotal experiments completed: {len(all_results)}')

## 13. Save Results

In [ ]:
save_json([asdict(r) for r in all_results], f'{SAVE_DIR}/results/E8_length_cue_results.json')
print(f'Results saved: {SAVE_DIR}/results/E8_length_cue_results.json')

save_json(traces_log, f'{SAVE_DIR}/results/E8_traces_sample.json')
print(f'Trace samples saved: {SAVE_DIR}/results/E8_traces_sample.json')

## 14. Analysis

In [ ]:
df = pd.DataFrame([asdict(r) for r in all_results])

# Pivot table
pivot = df.pivot_table(
    values='is_correct',
    index=['c', 'cue_condition'],
    columns='L',
    aggfunc='mean'
)

print('='*70)
print('E8 RESULTS: ACCURACY BY CONDITION')
print('='*70)
print((pivot * 100).round(1))
print('='*70)

In [ ]:
# Detailed table with L effect
print('\n' + '='*80)
print('DETAILED RESULTS')
print('='*80)
print(f'{"c":>5} {"Cue":>10} {"L=5":>8} {"L=10":>8} {"L=15":>8} {"L=20":>8} {"L Effect (20-5)":>15}')
print('-'*80)

for c in CORRUPTION_LEVELS:
    for cue in CUE_CONDITIONS:
        accs = {}
        for L in LENGTHS:
            mask = (df['c'] == c) & (df['cue_condition'] == cue) & (df['L'] == L)
            accs[L] = df[mask]['is_correct'].mean()
        
        l_effect = accs[20] - accs[5]
        effect_str = f'{l_effect*100:+.1f}pp'
        
        print(f'{c:>5.1f} {cue:>10} {accs[5]*100:>7.1f}% {accs[10]*100:>7.1f}% {accs[15]*100:>7.1f}% {accs[20]*100:>7.1f}% {effect_str:>15}')
    print('-'*80)

In [ ]:
# Key hypothesis tests
print('\n' + '='*70)
print('HYPOTHESIS TESTS')
print('='*70)

# H1: Cue-present is L-insensitive
print('\n1. CUE-PRESENT: Is performance L-insensitive?')
for c in CORRUPTION_LEVELS:
    mask_present = (df['cue_condition'] == 'present') & (df['c'] == c)
    acc_5 = df[(mask_present) & (df['L'] == 5)]['is_correct'].mean()
    acc_20 = df[(mask_present) & (df['L'] == 20)]['is_correct'].mean()
    diff = acc_20 - acc_5
    
    print(f'  c={c}: L=5 ({acc_5*100:.1f}%) → L=20 ({acc_20*100:.1f}%), diff = {diff*100:+.1f}pp')
    if abs(diff) < 0.05:
        print(f'    ✓ L-insensitive (cue dominates)')
    else:
        print(f'    △ Some L sensitivity detected')

# H2: Cue-absent shows L effect (redundancy)
print('\n2. CUE-ABSENT: Does L matter (redundancy)?')
for c in CORRUPTION_LEVELS:
    mask_absent = (df['cue_condition'] == 'absent') & (df['c'] == c)
    acc_5 = df[(mask_absent) & (df['L'] == 5)]['is_correct'].mean()
    acc_20 = df[(mask_absent) & (df['L'] == 20)]['is_correct'].mean()
    diff = acc_20 - acc_5
    
    print(f'  c={c}: L=5 ({acc_5*100:.1f}%) → L=20 ({acc_20*100:.1f}%), diff = {diff*100:+.1f}pp')
    if diff > 0.1:
        print(f'    ✓ Strong L effect (redundancy matters)')
    elif diff > 0.05:
        print(f'    △ Moderate L effect')
    else:
        print(f'    ✗ Weak or no L effect')

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, c in enumerate(CORRUPTION_LEVELS):
    ax = axes[idx]
    
    for cue in CUE_CONDITIONS:
        accs = []
        for L in LENGTHS:
            mask = (df['c'] == c) & (df['cue_condition'] == cue) & (df['L'] == L)
            accs.append(df[mask]['is_correct'].mean() * 100)
        
        marker = 'o' if cue == 'present' else 's'
        linestyle = '-' if cue == 'present' else '--'
        ax.plot(LENGTHS, accs, marker=marker, linestyle=linestyle, 
                label=f'Cue {cue}', linewidth=2, markersize=8)
    
    ax.set_xlabel('Trace Length (L)', fontsize=12)
    ax.set_ylabel('Accuracy (%)', fontsize=12)
    ax.set_title(f'c = {c} (corruption level)', fontsize=14)
    ax.set_xticks(LENGTHS)
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0, 100)

plt.suptitle('E8: Length × Cue Interaction (L=5,10,15,20)', fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/results/E8_length_cue_figure.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'Figure saved: {SAVE_DIR}/results/E8_length_cue_figure.png')

## 15. Summary

In [ ]:
# Compute summary statistics
summary_data = {}
for c in CORRUPTION_LEVELS:
    for cue in CUE_CONDITIONS:
        for L in LENGTHS:
            mask = (df['c'] == c) & (df['cue_condition'] == cue) & (df['L'] == L)
            key = f'c{c}_cue{cue}_L{L}'
            summary_data[key] = {
                'accuracy': float(df[mask]['is_correct'].mean()),
                'n_correct': int(df[mask]['is_correct'].sum()),
                'n_total': int(mask.sum())
            }

# Calculate L effects
l_effects = {}
for c in CORRUPTION_LEVELS:
    for cue in CUE_CONDITIONS:
        key = f'c{c}_cue{cue}'
        acc_5 = summary_data[f'c{c}_cue{cue}_L5']['accuracy']
        acc_20 = summary_data[f'c{c}_cue{cue}_L20']['accuracy']
        l_effects[key] = acc_20 - acc_5

summary = {
    'experiment': 'E8_length_cue_v2_FIXED',
    'date': EXPERIMENT_DATE,
    'n_problems': len(experiment_problems),
    'total_inferences': len(all_results),
    'conditions': {
        'lengths': LENGTHS,
        'corruption_levels': CORRUPTION_LEVELS,
        'cue_conditions': CUE_CONDITIONS
    },
    'results': summary_data,
    'l_effects': {k: float(v) for k, v in l_effects.items()},
    'interpretation': {
        'cue_present_l_insensitive': bool(all(abs(l_effects[f'c{c}_cuepresent']) < 0.1 for c in CORRUPTION_LEVELS)),
        'cue_absent_l_sensitive': bool(all(l_effects[f'c{c}_cueabsent'] > 0.05 for c in CORRUPTION_LEVELS))
    }
}

save_json(summary, f'{SAVE_DIR}/results/E8_summary.json')

print('='*70)
print('E8 EXPERIMENT COMPLETE (FIXED VERSION)')
print('='*70)
print(f'Date: {EXPERIMENT_DATE}')
print(f'Total experiments: {len(all_results)}')
print(f'\nL Effects (L=20 - L=5):')
for key, effect in l_effects.items():
    print(f'  {key}: {effect*100:+.1f}pp')
print(f'\nInterpretation:')
print(f'  Cue-present L-insensitive: {summary["interpretation"]["cue_present_l_insensitive"]}')
print(f'  Cue-absent L-sensitive: {summary["interpretation"]["cue_absent_l_sensitive"]}')
print(f'\nFiles saved to: {SAVE_DIR}')
print('='*70)